In [56]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd 
from fuzzywuzzy import fuzz
from rapidfuzz import process, fuzz
from geopy.geocoders import Nominatim
import time
import openpyxl
import concurrent.futures
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from fuzzywuzzy import fuzz
import time
import re
import random
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.namespace import Namespace
import rdflib
import openai
from rdflib.namespace import RDF, XSD

# this gets the tables from the list of all Belgian beers, and does some cleaning 

In [ ]:
url = "https://fr.wikipedia.org/wiki/Liste_de_bi%C3%A8res_belges"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

tables = soup.find_all("table", {"class": "wikitable"})

def remove_wikipedia_ref(text):
	return re.sub(r"\[\d+\]", "", text)

def remove_language_tags(text):
	return re.sub(r"\s*\((nl|en)\)", "", text)

def remove_trailing_commas(text):
	count_trailing = re.search(r",+$", text)
	if count_trailing:
		return text[:-len(count_trailing.group())]
	return text

def remove_pour_and_following_text(text): # need to adjust to remove the parenthesis
	if "pour" in text:
		return text.split("pour")[0][:-1]
	return text

dataframes = []
for i, table in enumerate(tables):
    headers = [header.text.strip() for header in table.find_all("th")]
    rows = []
    for row in table.find_all("tr")[1:]: # start at 1 skip header 
        cols = row.find_all(["td", "th"])
        cols = [col.text.strip() for col in cols]
        cols = [remove_wikipedia_ref(col) for col in cols]
        cols = [remove_trailing_commas(col) for col in cols]
        cols = [remove_language_tags(col) for col in cols]
        cols = [remove_pour_and_following_text(col) for col in cols]
        if cols:
            rows.append(cols)
    df = pd.DataFrame(rows, columns=headers)
    dataframes.append(df)
	
beers = pd.concat(dataframes, ignore_index=True)

In [ ]:
beers

# this gets the data from list of Belgian breweries, as well as some additional beer

In [ ]:
url = "https://fr.wikipedia.org/wiki/Liste_de_brasseries_belges"

reponse = requests.get(url)
soup = BeautifulSoup(reponse.text, "html.parser")

tables = soup.find_all("table", {"class": "wikitable"})

dataframes_breweries = []
for i, table in enumerate(tables):
    headers = [header.text.strip() for header in table.find_all("th")]
    rows = []
    for row in table.find_all("tr")[1:]:
        cols = row.find_all(["td", "th"])
        cols = [col.text.strip() for col in cols]
        cols = [remove_wikipedia_ref(col) for col in cols]
        cols = [remove_trailing_commas(col) for col in cols]
        cols = [remove_language_tags(col) for col in cols]
        cols = cols[:6]
        if cols:
            rows.append(cols)
    df = pd.DataFrame(rows, columns=headers)
    dataframes_breweries.append(df.iloc[:, :4])
    
breweries = pd.concat(dataframes_breweries, ignore_index=True)

# this block just removes some useless columns and rows
breweries = breweries.iloc[:250, :4]

# we need to align the breweries from both datasets with the format of one of the two
for this we will use the transformers -- sentence_transformers
or fuzzy thing, idk which one will work best




In [ ]:
def rename_by_similarity(df_source, col_source, df_reference, col_reference, threshold=70):
    ref_values = df_reference[col_reference].dropna().unique()
    df_source = df_source.copy()

    def get_best_match(value):
        if pd.isna(value) or value == "":
            return value
        match, score, i = process.extractOne(value, ref_values, scorer=fuzz.ratio)
        return match if score >= threshold else value # renaming if above threshold

    df_source[col_source] = df_source[col_source].apply(get_best_match)
    return df_source

# here i'm going to add some coordinates using geopy library

In [ ]:
new = rename_by_similarity(beers, "Brasserie", breweries, "Brasserie")

In [ ]:
parents = []

for index, row in breweries.iterrows():
	if "(" in row["Brasserie"]:
		parents.append(row["Brasserie"].split("(")[1][:-1])
	else:
		parents.append(None)

breweries["Parent"] = parents

In [ ]:
breweries["Brasserie"] = breweries["Brasserie"].str.split('(').str[0].str.strip()

# here i just parse a .txt file i created using excel 

In [ ]:
# this was created by hand in excel, just copied the Parent column and did some =IF(condition) function
# as well as manually modifying the values of the column that were not missing
with open ("parent.txt", "r") as f:
	parents_list = [line.strip() for line in f]

In [ ]:
# now to assign it to the breweries dataset
breweries["Parent"] = parents_list

In [ ]:
# here i remove all parentheses from the "Brasserie" column
for i, row in beers.iterrows():
    if "(" in row["Brasserie"]:
        beers.at[i, "Brasserie"] = row["Brasserie"].split("(")[0].strip()

In [ ]:
breweries.to_csv("breweries.csv", index=False)
new.to_csv("beers.csv", index=False)

# this part of the notebook is about somehow merging the right brewery row to the beer

In [ ]:
# a maybe inefficient approach is to look at the similarity once again, and match that
# turns out this is mostly solved with a left join on the "Brasserie" column
merged_df = beers.merge(breweries, on="Brasserie", how="left")

In [ ]:
# i can save as .csv later, first i will just save as .xlsx and add everything missing manually
# i have to use the openpyxl engine, otherwise i can't encode everything properly
# so all accents etc disappear, which is very annoying
merged_df.to_excel("output.xlsx", index=False, engine="openpyxl")

In [ ]:
beers = pd.read_csv("beers.csv")
breweries = pd.read_csv("breweries.csv")

In [ ]:
beer_names = beers["Bières"].tolist()
brewery_names = beers["Brasserie"].tolist()

In [ ]:
df = pd.read_excel("output.xlsx")

# big issue -> the % is being copied with .explode, use multi column

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
# df['Principales bières'] = df['Principales bières'].fillna("").apply(lambda x: x.split(", ") if x else [])
# df = df.explode('Principales bières')
# df.drop(columns=['Principales bières'], inplace=True)

# here i translate to english

In [ ]:
translation_map = {
    'Liège': 'Liege',
    'Brabant wallon': 'Walloon Brabant',
    'Limbourg': 'Limburg',
    'Anvers': 'Antwerp',
    'Brabant flamand': 'Flemish Brabant',
    'Luxembourg': 'Luxembourg',
    'Namur': 'Namur',
    'West-Flanders': 'West Flanders',
    'Flemish-Brabant': 'Flemish Brabant',
    'Hainaut': 'Hainaut',
    'Flandre-Orientale': 'East Flanders',
    'Flandre-Occidentale': 'West Flanders',
    'Antwerp': 'Antwerp',
    'Bruxelles': 'Brussels',
    'East-Flanders': 'East Flanders',
    'Brabant Wallon': 'Walloon Brabant',
    'Walloon Brabant': 'Walloon Brabant'
}

df["Province"] = df["Province"].apply(lambda x: translation_map.get(x, x))


In [ ]:
df['Province'].unique()

In [ ]:
df.drop_duplicates(subset= 'Bières', inplace=True)
df.drop(columns=['Principales bières'], inplace=True) # this column is not needed anymore, it's hard to use

In [ ]:
beer_names = df["Bières"].tolist()

# here i'm just replacing some names in the brewery column

In [ ]:
Brasserie = df["Brasserie"].tolist()
for i, x in enumerate(Brasserie):
	i = str(i)
	if "par" in i:
		Brasserie[i] = i.split("par")[1].strip()

df["Brasserie"] = Brasserie

# this gets the initial URLs needed for the data scraping

In [ ]:
# here i am going to build a list of search URLs to scrape data from untappd
# what this should allow me to do is fetch the next URL i need, that of the page of each individual beer
# given untappd closed their API, i have to scrape using Proxies, which is a bit slow

template = "https://untappd.com/search?q=X"
search_url = []

for i, beer in enumerate(beer_names):
	search_url.append((template.replace("X", beer).replace(" ", "+"), beer))

In [ ]:
untappd = pd.read_csv("untappd.csv")

In [ ]:
out = list(untappd.itertuples(index=False, name=None))
len(out)

# scraping untappd, Gemini was used to get this working, as explained in the report 

In [ ]:
def find_link_fuzzy(driver, target_text, threshold=70): # threshold is lower than the one we used before, it helps to find more matches
    links = driver.find_elements(By.TAG_NAME, "a")
    best_match = None
    best_score = 0
    for link in links:
        link_text = link.text.strip()
        if link_text:
            score = fuzz.ratio(target_text.lower(), link_text.lower())
            if score > best_score:
                best_score = score
                best_match = link
    if best_match and best_score >= threshold:
        return best_match, best_score
    else:
        return None, 0

# this part of the code is given the initial search URL and the target text, it will try to find the URL of the target text
def get_url_fuzzy(driver, url_tuple):
    driver.get(url_tuple[0])
    time.sleep(random.uniform(1,3)) # add some delay
    try:
        link, score = find_link_fuzzy(driver, url_tuple[1])
        if link:
            url = link.get_attribute("href")
            print(f"match found: '{link.text}' (score: {score})")
            return url
        else:
            print(f"match found for '{url_tuple[1]}'")
            return None
    except Exception as e:
        print(f"error: {e}")
        return None

def get_data(url):
    chrome_options = Options()
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    try:
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    except Exception as e:
        print(f"error: {e}")
        return None, None, None # return None values if driver could not be created
    try:
        driver.get(url)
        print(f"opened: {url}")
        # this part of the code handled the consent form of Untappd. 
        # it is always present, but needs human interaction to be closed
        # selenium can handle it, but it's a bit tricky
        try:
            consent_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Consent"]')
            consent_button.click()
        except Exception as e:
            print(f"cannot find form: {e}")

        time.sleep(random.uniform(1, 3))
        # this gets the rating of the beer
        # you can find the rating in the span element with the class "num"
		# the rating is inside parentheses, so you have to strip them
        try:
            rating_element = driver.find_element(By.CSS_SELECTOR, 'span.num')
            rating_text = rating_element.text
            rating = rating_text.strip('()')
        except Exception as e:
            print(f"error: {e}")
            rating = None
		# same for checkin 
        try:
            checkin_element = driver.find_element(By.CSS_SELECTOR, 'span.count')
            checkin_text = checkin_element.text
            checkin_count = checkin_text.replace(',', '')
        except Exception as e:
            print(f"error: {e}")
            checkin_count = None
		# this part of the code gets the description of the beer
        try:
            # sometimes the description is hidden behind a "Read More" button
			# this part of the code tries to click it
            try:
                read_more_button = WebDriverWait(driver, random.uniform(3, 5)).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.read-more.track-click'))
                )
                read_more_button.click()
            except:
                print("error clicking read more")

            description_element = driver.find_element(By.CSS_SELECTOR, 'div.beer-descrption-read-less')
            description_text = description_element.text
            description = re.sub(r'Show Less.*', '', description_text).strip()
        except Exception as e:
            print(f"error: {e}")
            description = None

    except Exception as e: # when the URL cannot be opened, simply return None 
        print(f"error: {e}")
        rating, checkin_count, description = None, None, None

    finally:
        driver.quit()

    return rating, checkin_count, description

chrome_options = Options() # user agent for my version of chrome
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
a
processed = {beer[0] for beer in out} # out as a global variable

for i in search_url:
    if i[1] in processed: # we can skip if already processed
        continue

    url = get_url_fuzzy(driver, i)
    if url:
        rating, checkin_count, description = get_data(url)
        print(f"{i[1]} done!")
        out.append((i[1], rating, checkin_count, description))
    else: # if no URL found, add None values
        print(f"no URL found: {i[1]}")
        out.append((i[1], None, None, None))

    time.sleep(random.uniform(1, 6)) # this adds some delay between requests, to avoid being blocked

driver.quit()

In [ ]:
untappd = pd.DataFrame(out, columns=["Bières", "Rating", "Check-ins", "Description"])

In [ ]:
untappd

In [ ]:
untappd.drop_duplicates(subset="Bières", inplace=True)

In [ ]:
untappd.to_csv("untappd.csv", index=False)

In [ ]:
len(out)

# now i need to merge the dataframes together, the one with Untappd data, and the big dataset 

In [ ]:
df = df.merge(untappd, on="Bières", how="left")

# adding coordinates 


In [ ]:
geolocator = Nominatim(user_agent="myGeocoder", timeout=3) # have to try using different timeout values

coordinates = []

for index, row in df.iterrows():
    brewery = row["Brasserie"]
    commune = row["Commune"]
    province = row["Province"]

    retries = 3
    for i in range(retries):
        try:
            location = geolocator.geocode(brewery) # first we try getting coords of the brewery (most precise)
            if location:
                coordinates.append((location.latitude, location.longitude))
                break # when found coordinates break loop
            else:
                location = geolocator.geocode(commune) # moving onto commune
                if location:
                    coordinates.append((location.latitude, location.longitude))
                    break
                else:
                    coordinates.append(None) # nothing found, might add manually
                    break
        except Exception as e:
            print(f"Error processing {brewery}: {e}")
            time.sleep(5)
    
    print(index)

df["coordinates"] = coordinates

In [ ]:
df.to_csv("final.csv", index=False)

In [ ]:
for i in df.columns:
	col = df[i]
	if col.isnull().sum() > 0:
		print(i, col.isnull().sum())
	else:
		print(i, 0)

In [ ]:
translation_map = {
        'Bières': 'Beers',
        'Type': 'Type',
        'Teneur en alcool': 'Alcohol Content',
        'Brasserie': 'Brewery',
        'Commune': 'Municipality',
        'Province': 'Province',
        'Parent': 'Parent',
        'Rating': 'Rating',
        'Check-ins': 'Check-ins',
        'Description': 'Description'
    }

df = df.rename(columns=translation_map)

In [ ]:
# this function is used to convert the check-ins to a float
def convert_to_float(val):
    val = str(val).replace("+", "")
    if 'M' in val:
        return float(val.replace('M', '')) * 1000000
    return float(val)

df['Check-ins'] = df['Check-ins'].apply(convert_to_float)

In [ ]:
df['Latitude'] = df['coordinates'].apply(lambda x: x[0] if x else None)
df['Longitude'] = df['coordinates'].apply(lambda x: x[1] if x else None)
df.drop(columns=['coordinates'], inplace=True)

In [3]:
df = pd.read_csv("final.csv")

In [ ]:
df['Alcohol Content'] = df['Alcohol Content'].apply(lambda x: 0.0 if str(x) == "Unknown" else x)
df['Rating'] = df['Rating'].apply(lambda x: 0 if str(x) == "Unknown" else x)
df['Check-ins'] = df['Check-ins'].apply(lambda x: 0 if str(x) == "Unknown" else x)
df['Latitude'] = df['Latitude'].apply(lambda x: 0.0 if str(x) == "Unknown" else x)
df['Longitude'] = df['Longitude'].apply(lambda x: 0.0 if str(x) == "Unknown" else x)


In [ ]:
df['Beers'] = df['Beers'].astype(str)
df['Type'] = df['Type'].astype(str)
df['Brewery'] = df['Brewery'].astype(str)
df['Municipality'] = df['Municipality'].astype(str)
df['Province'] = df['Province'].astype(str)
df['Parent'] = df['Parent'].astype(str)
df['Description'] = df['Description'].astype(str)

In [4]:
df.dtypes

Beers               object
Type                object
Alcohol Content    float64
Brewery             object
Municipality        object
Province            object
Parent              object
Rating               int64
Check-ins            int64
Description         object
Latitude           float64
Longitude          float64
dtype: object

In [6]:
df.isnull().sum()

Beers              0
Type               0
Alcohol Content    0
Brewery            0
Municipality       0
Province           0
Parent             0
Rating             0
Check-ins          0
Description        0
Latitude           0
Longitude          0
dtype: int64

# using GPT to translate some stuff, and normalize columns 

In [ ]:
client = openai.OpenAI(api_key="")

def translate_and_normalize(text):
    """
    Translates the beer type from French to English and normalizes it.
    """
    prompt = f"""
    Translate the following beer types from French to English and normalize the names:"{text}"

    Example normalizations:
    - "bière brune" becomes "Brown Ale"
    - "bière blonde, pils" becomes "Blonde Ale, Pilsner"

    Return a cleaned, comma-separated list of the translated and normalized beer types.
    """

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in beer taxonomy."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    return response.choices[0].message.content.strip()

"Blonde Ale, Pilsner, Brown Ale"


In [ ]:
openai.api_key = ''

def get_keywords(text):
    prompt = f"""
    Extract some precise keywords related to beer profiles from the following description:"{text}"

    For example, if you are given: "This beer is a Belgian-style ale with a fruity aroma and a hint of spices."
    You should return: "Belgian-style ale, fruity aroma, hint of spices"
    
    Return a cleaned, comma-separated list of keywords.
    """

    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in extracting precise keywords."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    return response.choices[0].message.content.strip()

In [11]:
df['Type'] = df['Type'].apply(translate_and_normalize)

In [45]:
df['Description'] = df['Description'].apply(get_keywords)

# these are to correct issues with the LLM translation 

In [27]:
df["Type"] = df["Type"].str.split("→").str.get(1).fillna(df["Type"])
df["Type"] = df["Type"].str.split("->").str.get(1).fillna(df["Type"])
df["Type"] = df["Type"].str.split("translates to").str.get(1).fillna(df["Type"])
df["Type"] = df["Type"].str.replace("\"", "")

In [47]:
df.to_csv("final.csv", index=False)

# the replacement below is very important, because otherwise the graph simply cannot be parsed

In [ ]:
df['Beers'] = df['Beers'].str.replace("'", "_", regex=False)
df['Description'] = df['Description'].str.replace("'", "_", regex=False)
df['Brewery'] = df['Brewery'].str.replace("'", "_", regex=False)
df['Municipality'] = df['Municipality'].str.replace("'", "_", regex=False)
df['Brewery'] = df['Brewery'].str.replace("!", "_", regex=False)
df['Beers'] = df['Beers'].str.replace("%", "_", regex=False)

In [79]:
df

,Beers,Type,Alcohol Content,Brewery,Municipality,Province,Parent,Rating,Check-ins,Description,Latitude,Longitude
0,3 Schténg,"Top Fermentation, Old Flemish Brown",6.0,Brasserie Grain d'Orge,Plombières,Liege,Unknown,3,3652,Unknown,50.737732,5.961113
1,IV Saison,Blonde Saison,6.5,Brasserie de Jandrain-Jandrenouille,Orp-Jauche,Walloon Brabant,Unknown,3,11522,"blonde beer, 100% barley malt, unfiltered, unp...",50.677482,4.977243
2,V Cense,"Belgian Ale, Specialty",7.5,Brasserie de Jandrain-Jandrenouille,Orp-Jauche,Walloon Brabant,Unknown,3,4705,"ambrée, 100% malt d'orge, non filtrée, non pas...",50.677482,4.977243
3,VI Wheat,"Belgian Ale, Witbier",6.0,Brasserie de Jandrain-Jandrenouille,Orp-Jauche,Walloon Brabant,Unknown,3,6579,"malt de froment, malt d'orge, non filtrée, non...",50.677482,4.977243
4,7-PK,Belgian Blonde Ale,7.0,Brasserie Anders!,Halen,Limburg,Unknown,3,8801,"blond haverbier, 7 procent alcohol, kruidige a...",52.958748,5.723328
...,...,...,...,...,...,...,...,...,...,...,...,...
1850,Zulte,Brown Ale,8.0,Alken-Maes,Unknown,Unknown,Unknown,0,0,Unknown,50.879329,5.307086
1851,Zwalmse Tripel,Tripel,10.0,De Proefbrouwerij,Lochristi,West Flanders,Unknown,3,933,Unknown,51.115254,3.816097
1852,Zwarte Pol,Stout,7.1,Brasserie Inter-Pol,Houffalize,Luxembourg,Unknown,3,401,Unknown,50.132376,5.788693
1853,Zwet.be,Stout,8.4,De Proefbrouwerij,Lochristi,West Flanders,Unknown,0,0,Unknown,51.115254,3.816097


In [81]:
g = Graph()

EX = Namespace("http://example.org/")
SCHEMA = Namespace("http://schema.org/")

g.bind("ex", EX)
g.bind("rdf", RDF)
g.bind("xsd", XSD)
g.bind("schema", SCHEMA)

# here we have to define the properties we are going to use for the graph
EX_ownedBy = EX.ownedBy # parent company
EX_brewedBy = EX.brewedBy
EX_locatedInMunicipality = EX.locatedInMunicipality
EX_isContainedInProvince = EX.isContainedInProvince
EX_hasLocation = EX.hasLocation
EX_alcoholPercentage = EX.alcoholPercentage
EX_beerRating = EX.beerRating
EX_beerPopularity = EX.beerPopularity
EX_beerDescription = EX.beerDescription
EX_beerType = EX.beerType

for idx, row in df.iterrows():
    beer_name = row["Beers"]
    brewery_name = row["Brewery"]
    municipality = row["Municipality"]
    province = row["Province"]
    parent_company = row["Parent"]

	# i'm using a regex, all it does it replace spaces with underscores
    # you could also use .replace(" ", "_") but this is more flexible as it accounts for multiple spaces
    beer_uri = EX[re.sub(r"\s+", "_", beer_name)]
    brewery_uri = EX[re.sub(r"\s+", "_", brewery_name)]
    municipality_uri = EX[re.sub(r"\s+", "_", municipality)]
    province_uri = EX[re.sub(r"\s+", "_", province)]
    location_uri = EX[f"location_{row['Latitude']}_{row['Longitude']}"]

    # need to check if there is a parent company or not, most of the time there isn't
    if parent_company != "Unknown":
        parent_company_uri = EX[re.sub(r"\s+", "_", parent_company)]
        g.add((brewery_uri, EX_ownedBy, parent_company_uri))
        g.add((parent_company_uri, RDF.type, SCHEMA.Organization)) # parent company as an organization, but it could be a brewery
        g.add((parent_company_uri, SCHEMA.name, Literal(parent_company, datatype=XSD.string)))

	# define some classes
    g.add((beer_uri, RDF.type, SCHEMA.BreweryProduct))
    g.add((brewery_uri, RDF.type, SCHEMA.Brewery))
    g.add((municipality_uri, RDF.type, SCHEMA.City))
    g.add((province_uri, RDF.type, SCHEMA.AdministrativeArea))
    g.add((location_uri, RDF.type, SCHEMA.GeoCoordinates)) # and we use GeoCoordinates

    g.add((beer_uri, SCHEMA.name, Literal(beer_name, datatype=XSD.string)))

    alcohol_content = float(str(row["Alcohol Content"]).replace(",", "."))
    g.add((beer_uri, EX_alcoholPercentage, Literal(alcohol_content, datatype=XSD.float)))

    beer_rating = float(row["Rating"])
    g.add((beer_uri, EX_beerRating, Literal(beer_rating, datatype=XSD.float)))

    beer_popularity = int(row["Check-ins"])
    g.add((beer_uri, EX_beerPopularity, Literal(beer_popularity, datatype=XSD.integer)))

    # here we just need to handle the commas, and split the attributes 
    descriptions = str(row["Description"]).split(',')
    for description in descriptions:
        description = description.strip()
        if description: 
            g.add((beer_uri, EX_beerDescription, Literal(description, datatype=XSD.string)))
            
    types = str(row["Type"]).split(',')
    for type in types:
        type = type.strip()
        if type: 
            g.add((beer_uri, EX_beerType, Literal(type, datatype=XSD.string)))

    g.add((brewery_uri, SCHEMA.name, Literal(brewery_name, datatype=XSD.string)))
    g.add((municipality_uri, SCHEMA.name, Literal(municipality, datatype=XSD.string)))
    g.add((province_uri, SCHEMA.name, Literal(province, datatype=XSD.string)))

	# here add some relationships that we created
    g.add((beer_uri, EX_brewedBy, brewery_uri))
    g.add((brewery_uri, EX_locatedInMunicipality, municipality_uri))
    g.add((municipality_uri, EX_isContainedInProvince, province_uri))
    g.add((beer_uri, EX_hasLocation, location_uri))

    g.add((location_uri, SCHEMA.latitude, Literal(row["Latitude"], datatype=XSD.float)))
    g.add((location_uri, SCHEMA.longitude, Literal(row["Longitude"], datatype=XSD.float)))

g.serialize("beers_graph_with_parent_company.ttl", format="turtle")

print("saved as: beers_graph_with_parent_company.ttl")

saved as: beers_graph_with_parent_company.ttl


# here i'm going to directly modify the .ttl file to remove % and ' and other bad stuff 

In [82]:
# maybe it isn't the best approach, but it works well enough for this graph 
with open("beers_graph_with_parent_company.ttl", "r") as f:
	data = f.read()
	data = re.sub(r"['!%]", "", data)
	
with open("beers_graph_with_parent_company.ttl", "w") as f:
	f.write(data)

# return top 10 highest alcohol % 

In [ ]:
query = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <http://schema.org/>
    SELECT ?beerName ?alcoholPercentage
    WHERE {
        ?beer a schema:BreweryProduct ;
               schema:name ?beerName ;
               ex:alcoholPercentage ?alcoholPercentage .
    }
    ORDER BY DESC(?alcoholPercentage)
    LIMIT 10
"""

results = g.query(query)

for row in results:
    print(f"{row.beerName}: {row.alcoholPercentage}")


Top 10 Beers by Alcohol Percentage:
Black Damnation V (Double Black): 26.0
Cuvée du Flo Ambrée: 15.0
Black Albert: 13.0
Black Damnation I: 13.0
Black Damnation III (Black Mes): 13.0
Black Damnation IV (Coffée Club): 13.0
Bush Prestige: 13.0
Bush 7: 13.0
La Cambre: 13.0
Cuvée du Château: 13.0


In [75]:
query = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <http://schema.org/>
    SELECT ?breweryName (COUNT(?beer) AS ?beerCount)
    WHERE {
        ?beer a schema:BreweryProduct ;
               ex:brewedBy ?brewery .
        ?brewery schema:name ?breweryName .
    }
    GROUP BY ?breweryName
    ORDER BY DESC(?beerCount)
    LIMIT 10
"""

results = g.query(query)

for row in results:
    print(f"{row.breweryName}: {row.beerCount} beers")

De Proefbrouwerij: 111 beers
Brasserie De Hoorn: 57 beers
Brasserie Den Herberg: 54 beers
Brasserie du Bocq: 54 beers
Brasserie Huyghe: 50 beers
Brasserie Strubbe: 41 beers
Brasserie Alvinne: 35 beers
Brasserie De Graal: 31 beers
Brasserie Haacht: 30 beers
Alken-Maes: 28 beers


In [70]:
query = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <http://schema.org/>
    SELECT (AVG(?alcoholPercentage) AS ?averageAlcoholPercentage)
    WHERE {
        ?beer a schema:BreweryProduct ;
               ex:alcoholPercentage ?alcoholPercentage .
    }
"""

# Execute the query
results = g.query(query)

# Print the results
for row in results:
    print(f"Average Alcohol Percentage: {row.averageAlcoholPercentage}")

Average Alcohol Percentage: 6.795463864684773


In [76]:
query = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <http://schema.org/>
    SELECT ?breweryName (MAX(?alcoholPercentage) AS ?maxAlcoholPercentage)
    WHERE {
        ?beer a schema:BreweryProduct ;
               ex:brewedBy ?brewery ;
               ex:alcoholPercentage ?alcoholPercentage .
        ?brewery schema:name ?breweryName .
    }
    GROUP BY ?breweryName
    ORDER BY DESC(?maxAlcoholPercentage)
    LIMIT 10
"""

# Execute the query
results = g.query(query)

# Print the results
print("Top 10 Breweries with Highest Alcohol Percentage Beers:")
for row in results:
    print(f"{row.breweryName}: Highest Alcohol Percentage = {row.maxAlcoholPercentage}")


Top 10 Breweries with Highest Alcohol Percentage Beers:
Brasserie Deca: Highest Alcohol Percentage = 26.0
Brasserie du Flo: Highest Alcohol Percentage = 15.0
Brasserie Het Anker: Highest Alcohol Percentage = 13.0
Brasserie Affligem: Highest Alcohol Percentage = 13.0
Brasserie Slaghmuylder: Highest Alcohol Percentage = 13.0
Brasserie Van Honsebrouck: Highest Alcohol Percentage = 13.0
Brasserie Dubuisson: Highest Alcohol Percentage = 13.0
Brasserie La Binchoise: Highest Alcohol Percentage = 12.0
Brasserie Lefebvre: Highest Alcohol Percentage = 12.0
De Proefbrouwerij: Highest Alcohol Percentage = 12.0


In [77]:
query = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <http://schema.org/>
    SELECT ?provinceName (COUNT(DISTINCT ?brewery) AS ?breweryCount)
    WHERE {
        ?brewery a schema:Brewery ;
                 ex:locatedInMunicipality ?municipality .
        ?municipality ex:isContainedInProvince ?province .
        ?province schema:name ?provinceName .
    }
    GROUP BY ?provinceName
    ORDER BY DESC(?breweryCount)
"""

# Execute the query
results = g.query(query)

# Print the results
print("Number of Breweries per Province:")
for row in results:
    print(f"{row.provinceName}: {row.breweryCount} breweries")

Number of Breweries per Province:
Hainaut: 33 breweries
Unknown: 31 breweries
West Flanders: 30 breweries
Flemish Brabant: 28 breweries
Luxembourg: 24 breweries
East Flanders: 24 breweries
Liege: 17 breweries
Antwerp: 17 breweries
Limburg: 14 breweries
Walloon Brabant: 11 breweries
Namur: 11 breweries
Brussels: 5 breweries


In [ ]:
from collections import Counter

query = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <http://schema.org/>
    SELECT ?provinceName ?beerType
    WHERE {
        ?beer a schema:BreweryProduct ;
              ex:beerType ?beerType ;
              ex:brewedBy ?brewery .
        ?brewery ex:locatedInMunicipality ?municipality .
        ?municipality ex:isContainedInProvince ?province .
        ?province schema:name ?provinceName .
    }
    ORDER BY ?provinceName ?beerType
"""

results = g.query(query)

province_beer_types = {}
for row in results:
    province = str(row.provinceName)
    beer_type = str(row.beerType)
    if province not in province_beer_types:
        province_beer_types[province] = []
    province_beer_types[province].append(beer_type)

province_type_counts = {}
for province, beer_types in province_beer_types.items():
    province_type_counts[province] = Counter(beer_types)

print("type of beer per province:")
for province, type_counts in province_type_counts.items():
    print(f"{province}: {type_counts}")

Beer Type Counts per Province:
Antwerp: Counter({'Blonde Ale': 19, 'Tripel': 14, 'Brown Ale': 12, 'Amber Ale': 9, 'Special': 6, 'High fermentation': 5, 'Fruity Ale': 4, 'Organic': 4, 'Stout': 4, 'Witbier': 4, 'Christmas Beer': 3, 'Pilsner': 3, 'Top-Fermented': 3, 'IPA': 2, 'Red Ale': 2, 'Spéciale': 2, 'Trappist': 2, '(Label Beer)': 1, 'Abbey Ale': 1, 'Amber': 1, 'Belgian Ale': 1, 'Belgian Blonde Ale': 1, 'Blonde': 1, 'Brown': 1, 'Dubbel': 1, 'Farmhouse Ale': 1, 'Golden Blonde': 1, 'High fermentation blonde': 1, 'India Pale Ale': 1, 'Label Beer': 1, 'Pale Ale': 1, 'Porter': 1, 'Rauchbier': 1, 'Special Red Ale': 1, 'Specialty Ale': 1, 'Top-Fermenting': 1, 'Top-fermented': 1, 'bottle refermentation': 1})
Brussels: Counter({'Gueuze': 14, 'Lambic': 7, 'Fruity': 3, 'Blonde Ale': 2, 'Brown Ale': 2, 'Stout': 2, 'Amber': 1, 'Amber Ale': 1, 'Farmhouse Ale': 1, 'Gueuze\nAmbrée': 1, 'High Fermentation Blonde': 1, 'High fermentation': 1, 'Lambic in English.': 1, 'Seasonal Beer': 1, 'Tripel': 1, 'Wi

In [84]:
for s, p, o in g.triples((None, EX.beerType, None)):
    print(f"Subject: {s}, Object: {o}")
    break #only print one.

Subject: http://example.org/3_Schténg, Object: Top Fermentation


In [74]:
query = """
    PREFIX ex: <http://example.org/>
    PREFIX schema: <http://schema.org/>
    SELECT ?breweryName (AVG(?beerRating) AS ?avgRating)
    WHERE {
        ?beer a schema:BreweryProduct ;
               ex:brewedBy ?brewery ;
               ex:beerRating ?beerRating .
        ?brewery schema:name ?breweryName .
    }
    GROUP BY ?breweryName
    ORDER BY DESC(?avgRating)
    LIMIT 5
"""

# Execute the query
results = g.query(query)

# Print the results
print("Top 5 Breweries with Highest Average Beer Rating:")
for row in results:
    print(f"{row.breweryName}: Average Rating = {row.avgRating}")

Top 5 Breweries with Highest Average Beer Rating:
Brasserie De Dolle Brouwers: Average Rating = 3.111111111111111
Brasserie de Jandrain-Jandrenouille: Average Rating = 3.0
Huisbrouwerij 't Pakhuis: Average Rating = 3.0
Huisbrouwerij t Pakhuis: Average Rating = 3.0
Brasserie de Battignies: Average Rating = 3.0


# this section is to get metrics about the graph 

In [93]:
num_triples = len(g) # this is just the number of triples in the graph

unique_nodes = set() # using set to get unique nodes
for s, p, o in g:
    unique_nodes.add(s)
    unique_nodes.add(o)
num_unique_nodes = len(unique_nodes)

degree_count = {} # here we compute the degree of each node
for s, p, o in g:
    degree_count[s] = degree_count.get(s, 0) + 1
    degree_count[o] = degree_count.get(o, 0) + 1
avg_degree = sum(degree_count.values()) / len(degree_count) # average it out

max_possible_edges = num_unique_nodes * (num_unique_nodes - 1) # here sparsity
sparsity = 1 - (num_triples / max_possible_edges) if max_possible_edges > 0 else 0

print(f"triples: {num_triples}")
print(f"nodes: {num_unique_nodes}")
print(f"avg degree: {avg_degree}")
print(f"sparsity: {sparsity}")

triples: 21880
nodes: 8959
avg degree: 4.884473713584105
sparsity: 0.9997273680668908


In [99]:
# idk why i didn't just print, works anyways 

missing = (df['Alcohol Content'] == 0).sum()
print(missing)
missing = (df['Rating'] == 0).sum()
print(missing)
missing = (df['Check-ins'] == 0).sum()
print(missing)
missing = (df['Latitude'] == 0).sum()
print(missing)
missing = (df['Longitude'] == 0).sum()
print(missing)

missing = (df['Parent'] == "Unknown").sum()
print(missing)
missing = (df['Description'] == "Unknown").sum()
print(missing)
missing = (df['Type'] == "Unknown").sum()

15
629
561
1
1
1684
1493
